In [1]:
import yaml
import os
from langchain_anthropic import ChatAnthropic
from sentence_transformers import SentenceTransformer
from matching_functions import find_closest_documents

model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
anthropic_key = config["anthropic_key"]
os.environ["ANTHROPIC_API_KEY"] = anthropic_key
chat = ChatAnthropic(temperature=0, model_name="claude-3-haiku-20240307")

<All keys matched successfully>


In [2]:
from wikipedia_functions import get_random_page, get_page_links, get_page_content, check_wikipedia_pages_existence

start_page = get_random_page(1)[0]
end_page = "United States"
end_page_full_content = get_page_content(end_page)
print(start_page)

The Tunnel (2019 film)


In [3]:
start_links = get_page_links(start_page)
checked_links = check_wikipedia_pages_existence(start_links)
valid_links = [link for link in start_links if checked_links[link]]

In [4]:
from chain_functions import get_crawler_chain, get_summarize_chain, get_explain_links_chain, get_broad_links_chain

crawler_chain = get_crawler_chain(chat)
summarize_chain = get_summarize_chain(chat)
explain_links_chain = get_explain_links_chain(chat)
broad_links_chain = get_broad_links_chain(chat)

In [5]:
from answer_functions import answer_broad_links, answer_crawler, answer_explain_links, answer_summarize

end_page_content = answer_summarize(summarize_chain, end_page_full_content)
print(end_page_content)

The United States is a federal republic with 50 states, a federal district, and several territories. It has a diverse population, with a history of European colonization, Native American displacement, and waves of immigration. The country has a large economy, advanced technology, and a prominent role in global affairs. The government has a presidential system with three branches, and the country has a long history of political and cultural development, including the American Revolution, Civil War, and rise as a superpower. The United States is known for its diversity, individualism, and global influence in areas like literature, arts, media, and sports.


In [6]:
current_page = start_page
current_links = valid_links
forbidden_links = [start_page.strip()]
for i in range(10):
    closest_links = find_closest_documents(model, end_page, end_page_content, current_links)
    broad_links = answer_broad_links(broad_links_chain, current_links, end_page, end_page_content)
    current_links = list(set([link for link in closest_links + broad_links if link in current_links]))
    reasonings = answer_explain_links(explain_links_chain, current_links, end_page, end_page_content)
    current_page = answer_crawler(crawler_chain, current_page, current_links, reasonings, end_page, end_page_content).strip()
    if current_page != end_page:
        forbidden_links.append(current_page)
        found_links = get_page_links(current_page)
        checked_links = check_wikipedia_pages_existence(found_links)
        current_links = [link for link in found_links if checked_links[link] and link not in forbidden_links]
    else:
        print(f"Page reached in {i} iterations!")
        break

<reasoning>
Based on the provided information, the link that seems most promising to reach the "United States" page is "Fandango Media". This is because Fandango is an American company operating in the entertainment industry, which has a direct connection to the United States. Exploring Fandango's operations, its role in the US movie industry, or the broader entertainment landscape in the country could provide a more straightforward path to the target page compared to the other options.

While the "Nordisk Film" and "Tank truck" links could potentially be used as stepping stones, they have a less direct connection to the United States and may require more navigational steps to reach the final destination. The "Fandango Media" link appears to be the most relevant and efficient choice to progress towards the "United States" page.
</reasoning>

<output>Fandango Media</output>
<reasoning>
Based on the information provided, the most useful link to get to the "United States" page would be "U